# Problem Description

In [36]:
!cat README.md


## The Objective

To build an efficient script that finds the closest airport to a given user
based on their geolocation and the geolocation of the airport.

## Data-sets

You will be provided with two data sets to consume:

 * `optd-sample-20161201.csv.gz` - A simplified version of a data set from Open Travel Data, containing geo-coordinates of major airports:
  * IATA airport code - a three-character identifier of global airports (first column)
  * Latitude and Longitude in floating point format (second and third columns, respectively)
  * The coordinates represent the location of the airport represented by the IATA code.

 * `sample_data.csv.gz` - Some sample input data for your script, containing:
  * A universally unique identifier (uuid) which identifies some end-user (first column)
  * Latitude and Longitude in floating point format (second and third columns, respectively)
  * For this challenge you need not concern yourself with the precise details of the uuid data and can sim

# Data

In [29]:
import pandas as pd
df_sample = pd.read_csv("sample_data.csv")
df_optd = pd.read_csv("optd-sample-20161201.csv")

In [35]:
display(df_sample.head())
display(df_sample.describe())

display(df_optd.head())
display(df_optd.describe())

,uuid,geoip_latitude,geoip_longitude
0,DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B,-37.833302,145.050003
1,DAEF2221-14BE-467B-894A-F101CDCC38E4,52.516701,4.666700
2,31971B3E-2F80-4F8D-86BA-1F2077DF36A2,35.685001,139.751404
3,1A29A45C-D560-43D8-ADAB-C2F0AD068FFE,44.840401,-0.580500
4,A6EC281B-B8EC-465A-8933-F127472DB0A3,51.963299,4.499700


,geoip_latitude,geoip_longitude
count,1000000.000000,1000000.000000
mean,41.464102,9.348279
std,22.786836,44.966706
min,-54.799999,-176.133301
25%,40.856300,2.103700
50%,48.787201,6.766700
75%,52.330576,12.483900
max,77.377502,178.483307


,iata_code,latitude,longitude
0,AAA,-17.352606,-145.509956
1,AAB,-26.693170,141.047800
2,AAC,31.073330,33.835830
3,AAE,36.822225,7.809167
4,AAF,29.729380,-85.028800


,latitude,longitude
count,6889.000000,6889.000000
mean,23.066441,-8.083803
std,28.634449,93.056961
min,-54.920000,-179.876830
25%,0.579211,-85.969780
50%,31.780020,-3.471656
75%,44.830000,66.000000
max,79.994720,179.975370


# Solution

In [ ]:
# %load solution.py
import getopt
import sys
import pandas as pd
from scipy.spatial import cKDTree

def joinByGPS(input_file, output_file, max_distance_in_degree, n_jobs=1):

    print(">>> The process started...")

    # This operation needs to be outside of this scope. It has to be loaded once.
    # Reading open traveling data in csv format
    df_db = pd.read_csv("optd-sample-20161201.csv")
    print(">>> The open traveling data loading")

    # Cleaning missing data
    df_db = df_db.dropna()
    print(">>> Cleaning missing data on open traveling data")

    # Selecting the coordinate over the dataframe
    db_lat_long = df_db[["latitude", "longitude"]]

    # A kd-tree is being created to efficient querying since our data is spatial data
    # In searching/querying, the average time complexity is O(log N)
    # In searching/querying, the worst case time complexity is O(N)
    print(">>> Creating Kd-tree")
    kdtree = cKDTree(db_lat_long.values)
    print(">>> Created Kd-tree")

    # Reading input file and creating a dataframe
    df_input = pd.read_csv(input_file)
    print(">>> The input file read " + input_file)

    # Cleaning missing data
    df_input = df_input.dropna()
    print(">>> Cleaning missing data on the input data")

    # Selecting the coordinate over the dataframe
    input_lat_long = df_input[["geoip_latitude", "geoip_longitude"]]

    # We are applying whole data on the tree. The overall time complexity will be O(M * log N).
    # M denoted the number of rows in the input file, sample_data.csv.
    print(">>> Querying starting")
    d, idx = kdtree.query(input_lat_long.values,
                          k=1,
                          eps=0,
                          p=2,  # Euclidean distance
                          distance_upper_bound=max_distance_in_degree,
                          n_jobs=n_jobs)
    print(">>> Querying finished")


    # Selecting iata_code from the data, optd-sample-20161201.csv
    lhs_df = df_db["iata_code"]

    # Filtering the iata_code by index we collected the previous step
    lhs_df = lhs_df.loc[idx]

    # Filling empty string to get ride of unmatched fields value, NaN.
    n_null = lhs_df.isnull().sum()
    if n_null > 0:
        print (">>> Number of mismatching point : " + str(n_null))
        lhs_df = lhs_df.fillna("")
        print(">>> Mismatching fields replaced with empty string " + str(n_null))

    # After those operations above, we need to rearrange its index to start from 0 to M-1.
    lhs_df = lhs_df.reset_index(drop=True)

    # Selecting uuid from the data sample_data.csv
    rhs_df = df_input['uuid']

    # Creating a new dataframe
    print(">>> Starting concat operations")
    new_df = pd.concat([lhs_df, rhs_df], axis=1)
    print(">>> Completed concat operations")

    # Writing result to CSV file under sample_data directory
    new_df.to_csv(output_file, sep=',', index=False)

    print(">>> Done! please, check out the output file, " + output_file)

    return

def usage():
    print("solution.py -i <input file> -o <output file> -md <max distance in degree> -n <number of jobs>")
    return

def main():
    try:
        opts, args = getopt.getopt(sys.argv[1:], 'i:o:d:j:h', ['input=', 'output=', 'max_distance=', 'n_jobs=', 'help'])
    except getopt.GetoptError:
        usage()
        sys.exit(2)
    if len(opts) != 4:
        usage()
        sys.exit(3)
    input_file = ""
    output = ""
    max_distance = 0.01
    n_jobs = 1
    for opt, arg in opts:
        if opt in ('-h', '--help'):
            usage()
            sys.exit(2)
        elif opt in ('-i', '--input'):
            input_file = arg
        elif opt in ('-o', '--output'):
            output = arg
        elif opt in ('-d', '--max_distance='):
            max_distance = arg
        elif opt in ('-j', '--n_job='):
            n_jobs = arg
        else:
            usage()
            sys.exit(4)

    joinByGPS(input_file, output, float(max_distance), int(n_jobs))

if __name__ == "__main__":
    main()


# Test

In [25]:
!python3 solution.py -i sample_data.csv -o result.csv -d 0.1 -j 10

>>> The process started...
>>> The open traveling data loading
>>> Cleaning missing data on open traveling data
>>> Creating Kd-tree
>>> Created Kd-tree
>>> The input file read sample_data.csv
>>> Cleaning missing data on the input data
>>> Querying starting
>>> Querying finished
>>> Number of mismatching point : 676501
>>> Mismatching fields replaced with empty string 676501
>>> Starting concat operations
>>> Completed concat operations
>>> Done! please, check out the output file, result.csv


# Report

In [15]:
from IPython.display import IFrame
IFrame("Report.pdf", width=1000, height=600)